In [ ]:
import os
import numpy as np
import pandas as pd
from flask import Flask, request
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
from collections.abc import Iterable
import pickle
from flask_cors import CORS
import math

app = Flask(__name__)
CORS(app)

@app.route('/')
def hello():
    return "Hello An!"


def weightedRating(x):
    v = x['Rating']
    R = x['AggregatedRating']
    return (v / (v + 51.0) * R) + (51.0 / (51.0 + v) * 4.56)

categoryBasedRecommender = pickle.load(open('categoryBasedRecommender.pkl', 'rb'))
metadataBasedRecommender = pickle.load(open('metadataBasedRecommender.pkl', 'rb'))

datasetDumpAlgolia = pickle.load(open('datasetDump.pkl', 'rb'))


# weighted popularity model
#qualifiedRecipes = pickle.load(open('qualifiedRecipes.pkl', 'rb'))


qualifiedRecipes = pickle.load(open('qualifiedRecipesCollaborative.pkl', 'rb'))
qualifiedRecipes['score'] = qualifiedRecipes.apply(weightedRating, axis=1)
df_rating = pd.DataFrame(qualifiedRecipes.groupby(by = ['RecipeId'])['score'].agg('mean')) 

filtered_ratings = pickle.load(open('filtered_ratings.pkl', 'rb'))
collaborativeMatrix = filtered_ratings.pivot(index = 'AuthorId', columns ='RecipeId', values = 'Rating').fillna(0)

recipe_metadata = qualifiedRecipes[['RecipeId','Name','RecipeCategory']]
recipe_data = filtered_ratings.merge(recipe_metadata,on='RecipeId')
knnMatrix = recipe_data.pivot(index = 'RecipeId', columns ='AuthorId', values = 'Rating').fillna(0)
    

@app.route('/popularity')
def weightedRatingPopularity():
    return qualifiedRecipes.to_json()


# category - got the data in weightedPopularRecipes and now creating the route function
@app.route('/category', methods=['POST'])
def getCategoryWiseRecommendations():
    data = request.get_json()
    RecipeCategory = data['keyword']
    percentile = 0.85
    categorySpecificRecipes = categoryBasedRecommender[categoryBasedRecommender['Category'] == RecipeCategory]
    RatingCounts = categorySpecificRecipes[categorySpecificRecipes['Rating'].notnull()]['Rating'].astype('int')
    RatingAverages = categorySpecificRecipes[categorySpecificRecipes['AggregatedRating'].notnull()][
        'AggregatedRating'].astype('int')
    C = RatingAverages.mean()
    m = RatingCounts.quantile(percentile)

    qualified = categorySpecificRecipes[
        (categorySpecificRecipes['Rating'] >= m) & (categorySpecificRecipes['Rating'].notnull()) & (
            categorySpecificRecipes['AggregatedRating'].notnull())][
        ['RecipeId', 'Name', 'Rating', 'AggregatedRating', 'ReviewCount']]
    qualified['Rating'] = qualified['Rating'].astype('int')
    qualified['AggregatedRating'] = qualified['AggregatedRating'].astype('int')

    qualified['wr'] = qualified.apply(
        lambda x: (x['Rating'] / (x['Rating'] + m) * x['AggregatedRating']) + (m / (m + x['Rating']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(12)
    print(qualified.to_json())
    return qualified.to_json()


# metadata
termFrequency2 = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0.0, stop_words='english')
tfidMatrix2 = termFrequency2.fit_transform(metadataBasedRecommender['Metadata'])
cosineSimilarity2 = linear_kernel(tfidMatrix2, tfidMatrix2)

metadataBasedRecommender = metadataBasedRecommender.reset_index()
names2 = metadataBasedRecommender[['RecipeId', 'Name']]
indices2 = pd.Series(metadataBasedRecommender.index, index=metadataBasedRecommender['Name'])


@app.route('/metadata', methods=['POST'])
def getMetadataBasedRecommendations():
    data = request.get_json()
    title = data['keyword']
    idx = indices2[title]
    if isinstance(idx, Iterable):
        for i in idx:
            similarityScores = sorted(list(enumerate(cosineSimilarity2[i])), key=lambda x: x[1], reverse=True)[1:13]
            break
    else:
        similarityScores = sorted(list(enumerate(cosineSimilarity2[idx])), key=lambda x: x[1], reverse=True)[1:13]
    recipeIndices = [i[0] for i in similarityScores]
    return names2.iloc[recipeIndices].to_json()


# content with ratings considered
contentBasedRecommender = metadataBasedRecommender


@app.route('/content', methods=['POST'])
def getContentBasedRecommendations():
    data = request.get_json()
    title = data['keyword']
    idx = indices2[title]

    if isinstance(idx, Iterable):
        recipeDump = pd.DataFrame()
        for i in idx:
            cnt = 0
            similarityScores = sorted(list(enumerate(cosineSimilarity2[i])), key=lambda x: x[1], reverse=True)[1:26]
            recipeIndices = [i[0] for i in similarityScores]
            recipeSimilarity = [i[1] for i in similarityScores]
            
            contentBasedRecommendedRecipes = contentBasedRecommender.iloc[recipeIndices][
                ['RecipeId', 'Name', 'Rating', 'AggregatedRating']]
            ratingCounts = contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['Rating'].notnull()][
                'Rating'].astype('int')
            ratingAverages = \
            contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['AggregatedRating'].notnull()][
                'AggregatedRating'].astype('int')
            C = ratingAverages.mean()
            m = ratingCounts.quantile(0.60)
            contentBasedRecommendedRecipes['Similarity'] = recipeSimilarity
            qualifiedRecipes2 = contentBasedRecommendedRecipes[(contentBasedRecommendedRecipes['Rating'] >= m) & (
                contentBasedRecommendedRecipes['Rating'].notnull()) & (contentBasedRecommendedRecipes[
                                                                           'AggregatedRating'].notnull())]
            qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
            qualifiedRecipes2['AggregatedRating'] = qualifiedRecipes2['AggregatedRating'].astype('int').head(12)
            if cnt == 0:
                recipeDump = qualifiedRecipes2
            else:
                recipeDump.append(qualifiedRecipes2, ignore_index=True)
            cnt += 1
        recipeDump = recipeDump.sort_values('Similarity', ascending=False)
        recipeDump = recipeDump.reset_index()
        return recipeDump.head(12).to_json()
 
    else:
        similarityScores = sorted(list(enumerate(cosineSimilarity2[idx])), key=lambda x: x[1], reverse=True)[1:26]
        recipeIndices = [i[0] for i in similarityScores]
        recipeSimilarity = [i[1] for i in similarityScores]
        
        contentBasedRecommendedRecipes = contentBasedRecommender.iloc[recipeIndices][
            ['RecipeId', 'Name', 'Rating', 'AggregatedRating']]
        ratingCounts = contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['Rating'].notnull()][
            'Rating'].astype('int')
        ratingAverages = contentBasedRecommendedRecipes[contentBasedRecommendedRecipes['AggregatedRating'].notnull()][
            'AggregatedRating'].astype('int')
        C = ratingAverages.mean()
        m = ratingCounts.quantile(0.60)
        contentBasedRecommendedRecipes['Similarity'] = recipeSimilarity
        qualifiedRecipes2 = contentBasedRecommendedRecipes[
            (contentBasedRecommendedRecipes['Rating'] >= m) & (contentBasedRecommendedRecipes['Rating'].notnull()) & (
                contentBasedRecommendedRecipes['AggregatedRating'].notnull())]
        qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
        qualifiedRecipes2['AggregatedRating'] = qualifiedRecipes2['AggregatedRating'].astype('int')
        qualifiedRecipes2 = qualifiedRecipes2.sort_values(by='Similarity', ascending=False)
        qualifiedRecipes2 = qualifiedRecipes2.reset_index()
        return qualifiedRecipes2.head(12).to_json()

@app.route('/collaborative', methods=['POST'])
def getCollaborativeRecommendations():
    data = request.json
    if 'likedRecipeList' in data:
        likedRecipeList = data['likedRecipeList']
        return getCollaborativeRecommendations(likedRecipeList)  
    return pd.DataFrame().to_json()

def getCollaborativeRecommendations(recipe_ids):
    recommendItemLength = 4
    
    for recipe_id in recipe_ids:
        if recipe_id not in collaborativeMatrix.columns:
            # remove recipe that doesnt have more than {threshold} count of reviews from participating in correlation matching
            recipe_ids.remove(recipe_id)

    recommendRecipeIndices = []
    for recipe_id in recipe_ids:
        users_rating = collaborativeMatrix[recipe_id]
        similar_recipes = collaborativeMatrix.corrwith(users_rating, method='pearson')
        similar_recipes = pd.DataFrame(similar_recipes,columns=['correlation'])
        result_recipes = similar_recipes.join(df_rating['score']).sort_values(by='correlation', ascending=False)
        result_recipes = result_recipes[result_recipes['score'] > 4].sort_values(by = 'correlation', ascending = False) 
        recommendRecipeIndices.extend(result_recipes.iloc[1:recommendItemLength + 1].index.tolist())

    filtered_rows = pd.DataFrame(columns=(qualifiedRecipes.columns.tolist() + ['SourceRecipeId']))
    
    for k in range(len(recommendRecipeIndices)):
        for i in range(len(qualifiedRecipes)):
            if qualifiedRecipes.iloc[i]['RecipeId'] == recommendRecipeIndices[k]:
                filtered_rows = filtered_rows.append(qualifiedRecipes.iloc[i], ignore_index=True)
                filtered_rows.loc[filtered_rows.index[-1], 'SourceRecipeId'] = recipe_ids[math.floor(k / recommendItemLength)]
                break
    filtered_rows = filtered_rows.sort_values(by=['SourceRecipeId', 'score'], ascending=[True, False])
    print(filtered_rows)
    filtered_rows = filtered_rows.reset_index()
    returnValue = filtered_rows[['RecipeId', 'Name', 'Images', 'TotalTime', 'RecipeCategory', 'SourceRecipeId', 'score']]
    return returnValue.to_json()

#print(getCollaborativeRecommendations([56, 10744]))

@app.route('/knn', methods=['POST'])
def getKNNRecommendations():
    data = request.json
    if 'likedRecipeList' in data:
        likedRecipeList = data['likedRecipeList']
        return knn_recommender(likedRecipeList)  
    return pd.DataFrame().to_json()

def knn_recommender(recipe_ids, matrix = knnMatrix, n_recs = 10, n_result = 4):
    n_recs = n_result * len(recipe_ids)
    cf_knn_model= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
    cf_knn_model.fit(matrix)
    cf_recs = pd.DataFrame(columns=(qualifiedRecipes.columns.tolist() + ['SourceRecipeId', 'Distance']))
    
    for recipe_id in recipe_ids:
        recipe_index = process.extractOne(str(recipe_id),qualifiedRecipes['RecipeId'])[2]
        distances, indices = cf_knn_model.kneighbors(matrix.iloc[recipe_index,:].values.reshape(1,-1), n_neighbors=n_recs)
        recipe_rec_ids = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        iterNo = range(len(recipe_rec_ids))
        for i in iterNo:
            if i == n_result:
                break
            index_to_append = recipe_rec_ids[i][0]
            if not any(cf_recs['RecipeId'] == qualifiedRecipes.iloc[index_to_append]['RecipeId']):
                cf_recs = cf_recs.append(qualifiedRecipes.iloc[index_to_append], ignore_index=True)
                cf_recs.loc[cf_recs.index[-1], 'SourceRecipeId']  = recipe_id
                cf_recs.loc[cf_recs.index[-1], 'Distance']  = recipe_rec_ids[i][1]
            else:
                iterNo = iterNo + 1
    
    cf_recs = cf_recs[['RecipeId', 'Name', 'Images', 'TotalTime', 'RecipeCategory', 'SourceRecipeId', 'Distance']]
    return cf_recs.to_json()

#print(knn_recommender([56, 26499]))

@app.route('/datasetDump')
def getDatasetDump():
    return datasetDumpAlgolia.to_json()


if __name__ == "__main__":
    osPort = os.getenv("PORT")
    if osPort == None:
        port = 5000
    else:
        port = int(osPort)
    app.run(host='0.0.0.0', port=port)

# class ModelDataHolder:
#     def __init__(self, matrix)

C:\Users\CY\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.92:5000
Press CTRL+C to quit
127.0.0.1 - - [03/May/2024 22:15:18] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:15:18] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:15:18] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:15:19] "POST /category HTTP/1.1" 200 -


{"RecipeId":{"2483":5194,"1567":3596,"6254":11630,"5919":11209,"6489":11930,"2390":5024,"229":346,"6060":11397,"172":267,"2398":5033,"2004":4330,"380":558},"Name":{"2483":"Scalloped Potatoes and Ham","1567":"Potato Soup","6254":"Awful Potatoes","5919":"Creamy Potato Leek Soup","6489":"Lemon Roast Potatoes","2390":"Potato Spinach Soup","229":"Potatoes Tapas in Garlic Mayonnaise (Potatoes Aioli)","6060":"Carrot & Potato Soup","172":"Twice Baked Potatoes","2398":"Dancer's Potato Skins","2004":"Sweet Potato Casserole II","380":"German Potato-Cheese Soup"},"Rating":{"2483":283,"1567":196,"6254":81,"5919":74,"6489":48,"2390":31,"229":30,"6060":26,"172":26,"2398":25,"2004":24,"380":24},"AggregatedRating":{"2483":5,"1567":5,"6254":5,"5919":5,"6489":5,"2390":5,"229":5,"6060":5,"172":5,"2398":5,"2004":5,"380":5},"ReviewCount":{"2483":290.0,"1567":203.0,"6254":83.0,"5919":76.0,"6489":49.0,"2390":31.0,"229":31.0,"6060":27.0,"172":26.0,"2398":25.0,"2004":25.0,"380":24.0},"wr":{"2483":4.9741466286,"

127.0.0.1 - - [03/May/2024 22:17:27] "GET /datasetDump HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:17:32] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:17:32] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:17:32] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:17:32] "POST /category HTTP/1.1" 200 -


...

127.0.0.1 - - [03/May/2024 22:28:07] "GET /datasetDump HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:28:20] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:28:20] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:28:20] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:28:20] "POST /category HTTP/1.1" 200 -


{"RecipeId":{"2467":5170,"4383":9054,"1419":3370,"6141":11492,"352":519,"5910":11194,"5172":10273,"6228":11593,"2193":4667,"3284":7437,"6326":11722,"6581":12055},"Name":{"2467":"Pete's Scratch Pancakes","4383":"Leftover Mashed Potato Pancakes","1419":"French Toast II","6141":"Apple Butter","352":"Spicy Hash Browns - Homemade","5910":"Latkes (Potato Pancakes)","5172":"Thin Pancakes","6228":"Gingerbread Waffles","2193":"Egg-White French Toast","3284":"Corned Beef Hash","6326":"Hash Browns Casserole","6581":"Crepes"},"Rating":{"2467":699,"4383":131,"1419":84,"6141":59,"352":49,"5910":47,"5172":32,"6228":28,"2193":27,"3284":23,"6326":21,"6581":20},"AggregatedRating":{"2467":5,"4383":5,"1419":5,"6141":5,"352":5,"5910":5,"5172":5,"6228":5,"2193":5,"3284":5,"6326":5,"6581":5},"ReviewCount":{"2467":722.0,"4383":134.0,"1419":85.0,"6141":61.0,"352":50.0,"5910":50.0,"5172":34.0,"6228":28.0,"2193":30.0,"3284":23.0,"6326":23.0,"6581":20.0},"wr":{"2467":4.9871614682,"4383":4.9363441437,"1419":4.9053

127.0.0.1 - - [03/May/2024 22:29:02] "GET /datasetDump HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:29:04] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:29:04] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:29:04] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 22:29:04] "POST /category HTTP/1.1" 200 -


{"RecipeId":{"2467":5170,"4383":9054,"1419":3370,"6141":11492,"352":519,"5910":11194,"5172":10273,"6228":11593,"2193":4667,"3284":7437,"6326":11722,"6581":12055},"Name":{"2467":"Pete's Scratch Pancakes","4383":"Leftover Mashed Potato Pancakes","1419":"French Toast II","6141":"Apple Butter","352":"Spicy Hash Browns - Homemade","5910":"Latkes (Potato Pancakes)","5172":"Thin Pancakes","6228":"Gingerbread Waffles","2193":"Egg-White French Toast","3284":"Corned Beef Hash","6326":"Hash Browns Casserole","6581":"Crepes"},"Rating":{"2467":699,"4383":131,"1419":84,"6141":59,"352":49,"5910":47,"5172":32,"6228":28,"2193":27,"3284":23,"6326":21,"6581":20},"AggregatedRating":{"2467":5,"4383":5,"1419":5,"6141":5,"352":5,"5910":5,"5172":5,"6228":5,"2193":5,"3284":5,"6326":5,"6581":5},"ReviewCount":{"2467":722.0,"4383":134.0,"1419":85.0,"6141":61.0,"352":50.0,"5910":50.0,"5172":34.0,"6228":28.0,"2193":30.0,"3284":23.0,"6326":23.0,"6581":20.0},"wr":{"2467":4.9871614682,"4383":4.9363441437,"1419":4.9053

C:\Users\CY\AppData\Local\Temp\ipykernel_18108\81136705.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualifiedRecipes2['Rating'] = qualifiedRecipes2['Rating'].astype('int')
C:\Users\CY\AppData\Local\Temp\ipykernel_18108\81136705.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualifiedRecipes2['AggregatedRating'] = qualifiedRecipes2['AggregatedRating'].astype('int')
127.0.0.1 - - [03/May/2024 22:31:34] "POST /content HTTP/1.1" 200 -
C:\Users\CY\AppData\Local\Temp\ipykernel_18108\81136705.py:1